In [9]:
from utils import *
from model import SimNet
import paddle
import paddle.nn.functional as F
import pandas as pd


V = TextVectorizer()
V.load_vocab_from_json()

In [10]:
def get_model(model):
    model = paddle.Model(model)
    optimizer = paddle.optimizer.Adam(
    parameters=model.parameters(), learning_rate=5e-4)
    criterion = paddle.nn.CrossEntropyLoss()
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model


def predict(model, data_loader):
    labels, predictions = [], []
    logits = model.predict(data_loader)
    for batch in data_loader:
        labels.extend(batch[-1].tolist())
        
    for batch in logits[0]:
        batch = paddle.to_tensor(batch)
        probs = F.softmax(batch, axis=1)
        preds = paddle.argmax(probs, axis=1).numpy().tolist()
        predictions.extend(preds)
        
    return predictions, labels


def get_accu_pre_recall_f1(preds, labels): 
    
    tp, fp, tn, fn, right = 0, 0, 0, 0, 0
    
    for (i, j) in zip(preds, labels):
        if i == 1 and j == 1:
            tp += 1
            right += 1
        elif i == 1 and j == 0:
            fp += 1
        elif i == 0 and j == 0:
            tn += 1
            right += 1
        else:
            fn += 1
    
    A = round(right/len(preds), 3)
    
    try:
        P = round(tp / (tp + fp), 3)
    except:
        return [A, 'Nan', 'Nan', 'Nan']

    R = round(tp / (tp + fn), 3)
    F1 = round(2 * P * R / (P + R), 3)
    
    return [A, P, R, F1]

In [11]:
def do_train_and_evaluate(train_path, 
                          network, 
                          epochs="3", 
                          dev_path='dev.txt', 
                          test_path='test.txt', 
                          batch_size=64, 
                          save=False, 
                          save_dir='ckpt',
                          device="gpu", 
                          log_freq=500):
    
    paddle.set_device(device)
    
    train_set, dev_set, test_set = load_dataset([train_path, dev_path, test_path])
    trans_fn = get_trans_fn(V, include_seq_len=True)
    batchify_fn = get_batchify_fn(include_seq_len=True)
    train_loader = create_dataloader(train_set, trans_fn, batchify_fn)
    dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn)
    test_loader = create_dataloader(test_set, trans_fn, batchify_fn, shuffle=False)

    model = SimNet(
        network=network,
        vocab_size=len(V),
        num_classes=2)
    
    model = get_model(model)
    if save:
        model.fit(train_loader, dev_loader, epochs=epochs, batch_size=batch_size,
                  verbose=2, log_freq=log_freq, save_dir=save_dir)
    else:
        model.fit(train_loader, dev_loader, epochs=epochs, batch_size=batch_size,
                  verbose=2, log_freq=log_freq)
        
    preds, labels = predict(model, test_loader)
    accu, prec, recall, f1 = get_accu_pre_recall_f1(preds, labels)
    
    return [accu, prec, recall, f1]

In [12]:
from IPython.display import clear_output

out = []

for size in ['10k', '50k', '100k', '150k', 'full']:
    for net in ['bow', 'cnn', 'lstm', 'gru']:
        print(f"Size: {size}; Model: {net}")
        res = do_train_and_evaluate(f'./all_data/train_{size}.txt', net, 3, device="gpu")
        out.append(['base', 'None', net, size] + res)
        clear_output(wait=True)

        columns = ['TrainType', 'EditType', 'ClfModel', 'TrainSize', 'Accuracy', 'Precision', 'Recall', 'F1']
        df = pd.DataFrame(out, columns=columns)
        df.to_excel('base_model_stats.xlsx', index=False)

Size: full; Model: gru
The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/3
step  500/4063 - loss: 0.4331 - acc: 0.6655 - 29ms/step
step 1000/4063 - loss: 0.6085 - acc: 0.6902 - 28ms/step
step 1500/4063 - loss: 0.5816 - acc: 0.7033 - 28ms/step
step 2000/4063 - loss: 0.4939 - acc: 0.7102 - 28ms/step
step 2500/4063 - loss: 0.4486 - acc: 0.7158 - 28ms/step
step 3000/4063 - loss: 0.6081 - acc: 0.7212 - 28ms/step
step 3500/4063 - loss: 0.4941 - acc: 0.7259 - 28ms/step
step 4000/4063 - loss: 0.5927 - acc: 0.7290 - 28ms/step
step 4063/4063 - loss: 0.5926 - acc: 0.7295 - 28ms/step
Eval begin...
step 313/313 - loss: 0.5522 - acc: 0.7525 - 24ms/step
Eval samples: 20000
Epoch 2/3
step  500/4063 - loss: 0.4722 - acc: 0.7929 - 29ms/step
step 1000/4063 - loss: 0.4321 - acc: 0.7952 - 29ms/step
step 1500/4063 - loss: 0.4930 - acc: 0.7935 - 29ms/step
step 2000/4063 - loss: 0.4421 - acc: 0.7933 - 29ms/step
step 2500/4063 - loss: 0.3825 -

In [ ]:
from IPython.display import clear_output

out = []

for aug_type in ['reda', 'reda_ngram']:
    for size in ['10k', '50k', '100k', '150k', 'full']:
        for net in ['bow', 'cnn', 'lstm', 'gru']:
            print(f"aug_type:{aug_type}. Size: {size}; Model: {net}")
            res = do_train_and_evaluate(f'./all_data/train_{size}_aug_{aug_type}.txt', net, 3, device="gpu")
            out.append([aug_type, 'All', net, size] + res)
            clear_output(wait=True)

            columns = ['TrainType', 'EditType', 'ClfModel', 'TrainSize', 'Accuracy', 'Precision', 'Recall', 'F1']
            df = pd.DataFrame(out, columns=columns)
            df.to_excel('aug_model_stats.xlsx', index=False)

aug_type:reda. Size: 100k; Model: bow
The loss value printed in the log is the current step, and the metric is the average value of previous steps.
Epoch 1/3
step   500/16970 - loss: 0.4618 - acc: 0.6978 - 23ms/step
step  1000/16970 - loss: 0.5146 - acc: 0.7184 - 24ms/step
step  1500/16970 - loss: 0.3985 - acc: 0.7305 - 23ms/step
step  2000/16970 - loss: 0.5019 - acc: 0.7397 - 24ms/step
step  2500/16970 - loss: 0.5234 - acc: 0.7471 - 23ms/step
step  3000/16970 - loss: 0.4164 - acc: 0.7542 - 23ms/step
step  3500/16970 - loss: 0.5033 - acc: 0.7604 - 23ms/step
step  4000/16970 - loss: 0.4378 - acc: 0.7657 - 23ms/step
step  4500/16970 - loss: 0.4711 - acc: 0.7703 - 23ms/step
step  5000/16970 - loss: 0.3753 - acc: 0.7750 - 23ms/step
step  5500/16970 - loss: 0.3408 - acc: 0.7794 - 23ms/step
step  6000/16970 - loss: 0.4934 - acc: 0.7834 - 23ms/step
step  6500/16970 - loss: 0.3445 - acc: 0.7877 - 23ms/step
step  7000/16970 - loss: 0.2212 - acc: 0.7916 - 23ms/step
step  7500/16970 - loss: 0.301